**Objectif du projet**: Prédire si un client est éligible à un prêt.

**Les étapes du projet :**

**EDA (Analyse Exploratoire des Données) :** 
- prise en main du jeu de données, nettoyage
- analyse de corrélation des variables
- définition des variables pertinentes

=> output : dataframe propre avec les variables pertinentes pour la prédiction du prix du bien

**Base de données SQL :**
- prendre bdd nettoyée (nans, duplicates, outliers)
- garder les colonnes qui peuvent être utiles meme si c'est pas utile pour nos predictions (dans mon cas garder id pour sql)
- garder les données catégorielle (on les numérisera plus tard)

**Développer un programme d'intelligence artificielle :**
- choix et entrainement de modèles de prédiction
- évaluation et comparaison de ces modèles
- enregister le modèle le plus performant (pickle..)

=> output: retenir le modèle plus performant.

**Développement API (Flask) :**
- définir une application flask
- charger le modèle de prédiction retenu  

=> output: tester le modèle à partir d'une page web.

## EDA

Prise en main du  jeu de données: 
- collecte, nettoyage de données
- vérifications/transformations de types (int,float,datetime,string ..) + définition de nouvelles variables(date? ..)
- vérification des données: valeurs uniques(distinctes)/ nulles/ non renseignées / outliers ? (boxplot)...

### Importation des librairies

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import GridSearchCV

%matplotlib inline 

import warnings
warnings.filterwarnings('ignore')

### Importation des données

In [2]:
#loading the dataset
train = pd.read_csv(r'C:\Users\Asma\Documents\ExoSimplon\Sem15_Grand_Projet\Data\train_u6lujuX_CVtuZ9i.csv')
test = pd.read_csv(r'C:\Users\Asma\Documents\ExoSimplon\Sem15_Grand_Projet\Data\test_Y3wMUE5_7gLdaTN.csv')

In [3]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [4]:
test.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


=> Pas de colonne target = 'Loan_Status' pour le test set

In [5]:
print('Train shape : ', train.shape, '\nTest  shape : ', test.shape)

Train shape :  (614, 13) 
Test  shape :  (367, 12)


- Concaténer les deux dataframes train et test

In [6]:
#df = pd.concat([train, test])

In [7]:
#df.shape

In [8]:
# use .describe() to get more information on the dataset. 
train.describe().T

,count,mean,std,min,25%,50%,75%,max
ApplicantIncome,614.0,5403.459283,6109.041673,150.0,2877.5,3812.5,5795.00,81000.0
CoapplicantIncome,614.0,1621.245798,2926.248369,0.0,0.0,1188.5,2297.25,41667.0
LoanAmount,592.0,146.412162,85.587325,9.0,100.0,128.0,168.00,700.0
Loan_Amount_Term,600.0,342.000000,65.120410,12.0,360.0,360.0,360.00,480.0
Credit_History,564.0,0.842199,0.364878,0.0,1.0,1.0,1.00,1.0


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


### Vérification des doubons

In [10]:
train.duplicated().sum()

0

In [11]:
#train[train.duplicated()]

In [12]:
#train.drop_duplicates(inplace=True)

In [13]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


### Vérification et Gestion des données manquantes (NAN)

In [14]:
train.isna().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

- Remplacement de 'Gender'

In [15]:
train['Gender'].unique()

array(['Male', 'Female', nan], dtype=object)

In [16]:
train['Gender'].value_counts()

Male      489
Female    112
Name: Gender, dtype: int64

In [17]:
train.dropna(subset=['Gender'],inplace=True)
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [18]:
train['Gender'].isna().sum()

0

In [19]:
train.reset_index(drop=True) #drop=true permet de supprimer l'ancien index et garder que le réindexage

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
597,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
598,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
599,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [20]:
train['Gender'].isna().sum()

0

In [21]:
train.isna().sum()

Loan_ID               0
Gender                0
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       49
Property_Area         0
Loan_Status           0
dtype: int64

- Remplacement de 'Married'

In [22]:
train['Married'].unique()

array(['No', 'Yes', nan], dtype=object)

In [23]:
train['Married'] = train['Married'].fillna('No')

In [24]:
train['Married'].unique()

array(['No', 'Yes'], dtype=object)

In [25]:
train.isna().sum()

Loan_ID               0
Gender                0
Married               0
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       49
Property_Area         0
Loan_Status           0
dtype: int64

- Remplacement de 'Dependents'

In [26]:
train['Dependents'].unique()

array(['0', '1', '2', '3+', nan], dtype=object)

In [27]:
train.Dependents = train.Dependents.fillna('0')

rpl = {'0':'0', '1':'1', '2':'2', '3+':'3'}

train.Dependents = train.Dependents.replace(rpl).astype(int)

In [28]:
train.Dependents.unique()

array([0, 1, 2, 3])

In [29]:
train.isna().sum()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       49
Property_Area         0
Loan_Status           0
dtype: int64

- Remplacement de 'Self_Employed'

In [30]:
train['Self_Employed'].unique()

array(['No', 'Yes', nan], dtype=object)

In [31]:
train['Self_Employed'].value_counts()

No     491
Yes     78
Name: Self_Employed, dtype: int64

In [32]:
train['Self_Employed'] = train['Self_Employed'].fillna('No')

In [33]:
train['Self_Employed'].unique()

array(['No', 'Yes'], dtype=object)

In [34]:
train['Self_Employed'].value_counts()

No     523
Yes     78
Name: Self_Employed, dtype: int64

In [35]:
train.isna().sum()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       49
Property_Area         0
Loan_Status           0
dtype: int64

- Remplacement de 'Credit_History'

In [36]:
train['Credit_History'].unique()

array([ 1.,  0., nan])

In [37]:
train['Credit_History'].value_counts()

1.0    466
0.0     86
Name: Credit_History, dtype: int64

In [38]:
train['Credit_History'].isna().sum()

49

In [39]:
train.dropna(subset=['Credit_History'],inplace=True)

In [40]:
train.reset_index(drop=True) #drop=true permet de supprimer l'ancien index et garder que le réindexage

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
547,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
548,LP002979,Male,Yes,3,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
549,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
550,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [41]:
train['Credit_History'].isna().sum()

0

- Remplacement de 'LoanAmount'

In [42]:
train['LoanAmount'].unique()

array([ nan, 128.,  66., 120., 141., 267.,  95., 158., 168., 349.,  70.,
       109., 200., 114.,  17., 125.,  76., 133., 115., 104., 315., 116.,
       191., 122., 110.,  35.,  74., 106., 320., 100., 144., 184.,  80.,
        47., 134.,  96.,  88.,  44., 112., 286.,  97., 135., 180.,  99.,
       165., 258., 126., 312., 136., 172.,  81., 187., 113., 176., 111.,
       167.,  50., 210., 175., 131., 188.,  25., 137., 151., 160., 225.,
       216.,  94., 139., 152., 118., 185., 154., 259., 194., 102., 290.,
        84., 242., 129.,  30., 244., 600., 255.,  98., 275., 121.,  75.,
       182.,  63.,  87., 101., 495.,  73., 260., 108.,  48., 164., 170.,
        83.,  90., 166., 124.,  55.,  59., 127., 214., 240., 130.,  72.,
        60., 138.,  42., 280., 140., 155., 123., 201., 279., 192., 304.,
       150., 207., 436.,  78.,  54.,  89., 143.,  93., 105., 132., 480.,
        56., 300., 376.,  67., 117.,  71., 490., 173.,  46., 228., 308.,
       236., 570., 380., 296., 156., 103.,  45.,  6

In [43]:
#train['LoanAmount'].fillna(train['LoanAmount'].mode()[0], inplace=True)

In [44]:
train.dropna(subset=['LoanAmount'],inplace=True)

In [45]:
train.reset_index(drop=True) #drop=true permet de supprimer l'ancien index et garder que le réindexage

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
1,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
2,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
3,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
4,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
526,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
527,LP002979,Male,Yes,3,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
528,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
529,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [46]:
train['LoanAmount'].isna().sum()

0

- Remplacement de 'Loan_Amount_Term'

In [47]:
train['Loan_Amount_Term'].unique()

array([360., 120.,  nan, 180.,  60., 300., 480., 240.,  36.,  84.])

In [48]:
#train['Loan_Amount_Term'].fillna(train['Loan_Amount_Term'].mode()[0], inplace=True)

In [49]:
train.dropna(subset=['Loan_Amount_Term'],inplace=True)

In [50]:
train.reset_index(drop=True) #drop=true permet de supprimer l'ancien index et garder que le réindexage

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
1,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
2,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
3,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
4,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
513,LP002979,Male,Yes,3,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
514,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
515,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [51]:
train['Loan_Amount_Term'].isna().sum()

0

- Revérification des NaNs

In [52]:
train.isna().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [53]:
print('Train shape :', train.shape)

Train shape : (517, 13)


### Changer les types des données

In [54]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 517 entries, 1 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            517 non-null    object 
 1   Gender             517 non-null    object 
 2   Married            517 non-null    object 
 3   Dependents         517 non-null    int32  
 4   Education          517 non-null    object 
 5   Self_Employed      517 non-null    object 
 6   ApplicantIncome    517 non-null    int64  
 7   CoapplicantIncome  517 non-null    float64
 8   LoanAmount         517 non-null    float64
 9   Loan_Amount_Term   517 non-null    float64
 10  Credit_History     517 non-null    float64
 11  Property_Area      517 non-null    object 
 12  Loan_Status        517 non-null    object 
dtypes: float64(4), int32(1), int64(1), object(7)
memory usage: 54.5+ KB


In [55]:
train['Credit_History'] = train['Credit_History'].astype(int)

In [56]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 517 entries, 1 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            517 non-null    object 
 1   Gender             517 non-null    object 
 2   Married            517 non-null    object 
 3   Dependents         517 non-null    int32  
 4   Education          517 non-null    object 
 5   Self_Employed      517 non-null    object 
 6   ApplicantIncome    517 non-null    int64  
 7   CoapplicantIncome  517 non-null    float64
 8   LoanAmount         517 non-null    float64
 9   Loan_Amount_Term   517 non-null    float64
 10  Credit_History     517 non-null    int32  
 11  Property_Area      517 non-null    object 
 12  Loan_Status        517 non-null    object 
dtypes: float64(3), int32(2), int64(1), object(7)
memory usage: 52.5+ KB


In [57]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1,Urban,Y
5,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1,Urban,Y


### Créer une copie de mon df

In [58]:
new_train = train.copy()

# BDD SQL

### Convertion des variables categorielles en variables numériques

In [59]:
new_train.head(3)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1,Urban,Y


In [60]:
new_train['Married'] = new_train['Married'].map(dict(Yes=1, No=0))
test['Married'] = test['Married'].map(dict(Yes=1, No=0))

In [61]:
new_train['Self_Employed'] = new_train['Self_Employed'].map(dict(Yes=1, No=0))
test['Self_Employed'] = test['Self_Employed'].map(dict(Yes=1, No=0))

In [62]:
new_train['Education'].value_counts()

Graduate        410
Not Graduate    107
Name: Education, dtype: int64

In [63]:
new_train.head(2)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,LP001003,Male,1,1,Graduate,0,4583,1508.0,128.0,360.0,1,Rural,N
2,LP001005,Male,1,0,Graduate,1,3000,0.0,66.0,360.0,1,Urban,Y


In [64]:
to_numeric = {'Male': 0, 'Female': 1,
              'Graduate': 1, 'Not Graduate': 0,
              'Urban': 3, 'Semiurban': 2,'Rural': 1,
              'Y': 1, 'N': 0,
              '3+': 3}

In [65]:
# adding the new numeric values from the to_numeric variable to both datasets
new_train = new_train.applymap(lambda elem: to_numeric.get(elem) if elem in to_numeric else elem)
test = test.applymap(lambda elem: to_numeric.get(elem) if elem in to_numeric else elem)

In [66]:
new_train['Education'].value_counts()

1    410
0    107
Name: Education, dtype: int64

In [67]:
new_train['Education'].dtypes

dtype('int64')

In [68]:
obj = (new_train.dtypes == 'object')
print("Categorical variables:",len(list(obj[obj].index)))

Categorical variables: 1


In [69]:
new_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 517 entries, 1 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            517 non-null    object 
 1   Gender             517 non-null    int64  
 2   Married            517 non-null    int64  
 3   Dependents         517 non-null    int64  
 4   Education          517 non-null    int64  
 5   Self_Employed      517 non-null    int64  
 6   ApplicantIncome    517 non-null    int64  
 7   CoapplicantIncome  517 non-null    float64
 8   LoanAmount         517 non-null    float64
 9   Loan_Amount_Term   517 non-null    float64
 10  Credit_History     517 non-null    int64  
 11  Property_Area      517 non-null    int64  
 12  Loan_Status        517 non-null    int64  
dtypes: float64(3), int64(9), object(1)
memory usage: 56.5+ KB


In [70]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            367 non-null    object 
 1   Gender             356 non-null    float64
 2   Married            367 non-null    int64  
 3   Dependents         357 non-null    object 
 4   Education          367 non-null    int64  
 5   Self_Employed      344 non-null    float64
 6   ApplicantIncome    367 non-null    int64  
 7   CoapplicantIncome  367 non-null    int64  
 8   LoanAmount         362 non-null    float64
 9   Loan_Amount_Term   361 non-null    float64
 10  Credit_History     338 non-null    float64
 11  Property_Area      367 non-null    int64  
dtypes: float64(5), int64(5), object(2)
memory usage: 34.5+ KB


In [72]:
test['Gender '] = test['Gender'].astype(int)

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

### Vérification des outliers

### Supression des colonnes non utilisées pour la prédiction

In [ ]:
new_train.drop('Loan_ID', axis=1, inplace=True)
test.drop('Loan_ID',axis=1,inplace=True)

In [ ]:
new_train.head(2)

In [ ]:
test.head(2)

# DataViz

In [ ]:
# nunique() : pour trouver le nombre de valeurs uniques sur l’axe de la colonne

[print(col, train[col].nunique()) for col in train.columns if train[col].dtype=='object']

In [ ]:
sns.countplot(train['Gender']).set_title('Sexe');

In [ ]:
sns.countplot(train['Married']).set_title('Marrié ?');

In [ ]:
sns.countplot(train['Education']).set_title('Diplômé ?');

In [ ]:
train['Dependents'].value_counts()

In [ ]:
train['Dependents'].value_counts().index

In [ ]:
fig = plt.figure(figsize =(10, 7));
plt.pie(train['Dependents'].value_counts(), 
        labels=train['Dependents'].value_counts().index, 
        autopct='%1.1f%%',  
        startangle=90,
        textprops={'size': 'x-large'});
plt.legend(title = "Personnes à charge :", loc ="center right", bbox_to_anchor =(1.3, 0.7));

In [ ]:
sns.countplot(train['Self_Employed']).set_title('Travailleur indépendant ?');

In [ ]:
sns.countplot(train['Credit_History']).set_title('Crédit historique ?');

# Partie 2

- Analyse de corrélation des variables
- vérifier la multicolinéarité avec un heatmap (voir comment les variables indépendantes peuvent être corrélées)
- Définir les variables significatives
- transformer les variables continues

In [ ]:
#Matrice de corrélation

corr_df = train.corr().abs()

upp_mat = np.triu(train.corr())

plt.figure(figsize=(20, 15))
sns.heatmap(corr_df, annot=True, mask=upp_mat)
#plt.savefig("Matrice de corrélation.png")
plt.show()

### Multi Colinéarité

* Vérifier la Multicolinéarité des features

https://datascience.eu/fr/apprentissage-automatique/multicollinearite-2/

https://datascience.eu/fr/mathematiques-et-statistiques/multicollinearite/

In [ ]:
def Multicollinear_Features():
    corr = train.corr().abs()
    features = []
    correlations = []
    for idx, correlation in corr['price'].T.iteritems():
        if correlation >= .3 and idx != 'price':
            features.append(idx)
            correlations.append(correlation)
    corr_price_df = pd.DataFrame({'Correlations':correlations, 'Features': features})
    corr_price_df.sort_values(by='Correlations',ascending=False,inplace=True)
    corr_price_df.reset_index(drop=True,inplace=True)
      
    Multicollinear_Features = [] 
    for feature in corr:
        for idx, correlation in corr[feature].T.iteritems():
            if correlation >= .8 and idx != feature and corr['price'].loc[feature] >= corr['price'].loc[idx]:
                Multicollinear_Features.append({'Correlations':correlation,'Features':feature,'idx': idx})
    if len(Multicollinear_Features) > 0:
        MC_df = pd.DataFrame(Multicollinear_Features)
    else:
        MC_df = pd.DataFrame(columns=['Correlations', 'Features','idx'])
    MC_df.sort_values(by='Correlations',ascending=False,inplace=True)
    MC_df.reset_index(drop=True,inplace=True) 
    print('Multicollinear Features')
    display(MC_df)
    
    print('Correlations with Target')
    corr_price_df = corr_price_df.loc[~corr_price_df['Features'].isin(MC_df['idx'].to_list())]
    display(corr_price_df)
    return corr_price_df,MC_df

In [ ]:
corr_price_df, MC_df = Multicollinear_Features()

* Quels sont les meilleures variables , les plus significatives ?

utiliser stepwise_selection pour choisir les meilleurs features

https://towardsdatascience.com/feature-selection-techniques-in-regression-model-26878fe0e24e

https://en.wikipedia.org/wiki/Stepwise_regression

https://towardsdatascience.com/stepwise-regression-tutorial-in-python-ebf7c782c922

https://bookdown.org/max/FES/greedy-stepwise-selection.html

In [ ]:
#Nouveau df avec les features qu'on garde

values = []
cols = []
for col, value in df.iteritems():
    if (col in corr_price_df['Features'].to_list()):
        values.append(df[col])
        cols.append(col)
df1 = pd.concat(values,axis=1)
df1

In [ ]:
df1 = df[['price'] + df1.columns.tolist()]
df1

# Sauvegarde du nouveau df propre